# 🔄 Incremental Data Update v3

**Now checks an entire week of matches to catch any gaps.**

✅ Checks up to 20 pages of recent matches (~1 week per league)
✅ Uses match_id to skip already-known matches
✅ Handles Liga MX Apertura + Clausura

In [1]:
!pip install tls_client pandas -q
print('Ready!')

Ready!


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: C:\Users\Carlos\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
#@title 📤 Upload your existing data CSV
from google.colab import files
import pandas as pd

print('Upload your existing CSV:')
uploaded = files.upload()

filename = list(uploaded.keys())[0]
df_existing = pd.read_csv(filename)
df_existing['date'] = pd.to_datetime(df_existing['date'])

last_date = df_existing['date'].max()
existing_ids = set(df_existing['match_id'].astype(int).tolist())

print(f'\n✅ Loaded {len(df_existing):,} existing matches')
print(f'📅 Last match date: {last_date.strftime("%Y-%m-%d")}')
print(f'🆔 Known match IDs: {len(existing_ids):,}')

ModuleNotFoundError: No module named 'google'

In [ ]:
#@title Configuration { run: "auto" }
#@markdown ### Leagues
Premier_League = True #@param {type:"boolean"}
La_Liga = True #@param {type:"boolean"}
Bundesliga = True #@param {type:"boolean"}
Serie_A = True #@param {type:"boolean"}
Ligue_1 = True #@param {type:"boolean"}
Liga_MX = True #@param {type:"boolean"}

#@markdown ### Options
Pages_To_Check = 20 #@param {type:"slider", min:5, max:50, step:5}
Parallel_Workers = 5 #@param {type:"slider", min:2, max:10, step:1}

LEAGUES = {
    'Premier League': {'ids': [17]},
    'La Liga': {'ids': [8]},
    'Bundesliga': {'ids': [35]},
    'Serie A': {'ids': [23]},
    'Ligue 1': {'ids': [34]},
    'Liga MX': {'ids': [11621, 11620]},
}

enabled = {'Premier League': Premier_League, 'La Liga': La_Liga,
           'Bundesliga': Bundesliga, 'Serie A': Serie_A,
           'Ligue 1': Ligue_1, 'Liga MX': Liga_MX}
selected = {k: v for k, v in LEAGUES.items() if enabled.get(k, False)}
print(f'Selected: {len(selected)} leagues | {Pages_To_Check} pages per league')

In [ ]:
import time
import random
from concurrent.futures import ThreadPoolExecutor, as_completed
from threading import Lock
from tls_client import Session
from datetime import datetime, timedelta

class SessionPool:
    def __init__(self, size=5):
        self.sessions = [Session(client_identifier='firefox_120') for _ in range(size)]
        self.index = 0
        self.lock = Lock()
    def get(self):
        with self.lock:
            s = self.sessions[self.index % len(self.sessions)]
            self.index += 1
            return s

pool = SessionPool(Parallel_Workers)
BASE_URL = 'https://www.sofascore.com/api/v1'

def fetch_json(url, retries=2):
    full_url = f'{BASE_URL}{url}' if url.startswith('/') else url
    session = pool.get()
    for attempt in range(retries + 1):
        try:
            time.sleep(random.uniform(0.2, 0.5))
            r = session.get(full_url)
            if r.status_code == 200:
                return r.json()
            elif r.status_code == 403:
                time.sleep(3)
        except:
            time.sleep(1)
    return None

def convert_fractional(frac_str):
    try:
        if '/' in str(frac_str):
            n, d = map(int, str(frac_str).split('/'))
            return round(1 + (n / d), 3)
        return float(frac_str)
    except:
        return None

print('Session pool ready')

In [ ]:
def get_finished_matches_full_scan(tournament_id, existing_ids, max_pages):
    """
    FULL SCAN: Check ALL pages up to max_pages to find any missing matches.
    Does NOT stop early - ensures we catch any gaps in data.
    """
    # Get current season
    seasons_data = fetch_json(f'/unique-tournament/{tournament_id}/seasons')
    if not seasons_data or 'seasons' not in seasons_data:
        return []
    
    season_id = seasons_data['seasons'][0]['id']
    season_name = seasons_data['seasons'][0].get('name', 'Unknown')
    print(f'    Season: {season_name}')
    
    new_matches = []
    
    for page in range(max_pages):
        data = fetch_json(f'/unique-tournament/{tournament_id}/season/{season_id}/events/last/{page}')
        
        if not data or 'events' not in data:
            print(f'    Page {page}: No more data')
            break
        
        events = data.get('events', [])
        if not events:
            break
        
        page_new = 0
        page_known = 0
        for e in events:
            match_id = e.get('id')
            status_type = e.get('status', {}).get('type', '').lower()
            
            # Only finished matches
            if status_type not in ['finished', 'ended']:
                continue
            
            # Check if already known
            if match_id in existing_ids:
                page_known += 1
                continue
            
            # New match found!
            ts = e.get('startTimestamp', 0)
            new_matches.append({
                'match_id': match_id,
                'date': datetime.fromtimestamp(ts).strftime('%Y-%m-%d'),
                'timestamp': ts,
                'home_team': e.get('homeTeam', {}).get('name'),
                'home_team_id': e.get('homeTeam', {}).get('id'),
                'away_team': e.get('awayTeam', {}).get('name'),
                'away_team_id': e.get('awayTeam', {}).get('id'),
                'home_score': e.get('homeScore', {}).get('current'),
                'away_score': e.get('awayScore', {}).get('current'),
                'tournament_id': tournament_id,
            })
            page_new += 1
        
        print(f'    Page {page}: +{page_new} new, {page_known} known')
    
    return new_matches


def enrich_match(match):
    """Add stats, odds, H2H to a match."""
    match_id = match['match_id']
    
    # Stats
    stats_data = fetch_json(f'/event/{match_id}/statistics')
    if stats_data and 'statistics' in stats_data:
        for period in stats_data.get('statistics', []):
            pname = period.get('period', 'ALL').lower()
            for g in period.get('groups', []):
                for item in g.get('statisticsItems', []):
                    name = item.get('name', '').lower().replace(' ', '_')
                    key = name if pname == 'all' else f'{pname}_{name}'
                    match[f'{key}_home'] = item.get('home')
                    match[f'{key}_away'] = item.get('away')
    
    # Odds
    odds_data = fetch_json(f'/event/{match_id}/odds/1/all')
    if odds_data and 'markets' in odds_data:
        for market in odds_data.get('markets', []):
            mid = market.get('marketId')
            for choice in market.get('choices', []):
                name = choice.get('name', '')
                dec = convert_fractional(choice.get('fractionalValue', ''))
                if not dec:
                    continue
                if mid == 1:
                    if name == '1': match['odds_1x2_home'] = dec
                    elif name == 'X': match['odds_1x2_draw'] = dec
                    elif name == '2': match['odds_1x2_away'] = dec
                elif mid == 5:
                    if name.lower() == 'yes': match['odds_btts_yes'] = dec
                    elif name.lower() == 'no': match['odds_btts_no'] = dec
    
    # H2H
    h2h_data = fetch_json(f'/event/{match_id}/h2h/events')
    if h2h_data and 'events' in h2h_data:
        h2h_home = h2h_away = h2h_draws = 0
        for h2h in h2h_data.get('events', []):
            if h2h.get('startTimestamp', 0) >= match['timestamp']:
                continue
            if h2h.get('status', {}).get('type') != 'finished':
                continue
            winner = h2h.get('winnerCode')
            h2h_home_id = h2h.get('homeTeam', {}).get('id')
            if winner == 1:
                if h2h_home_id == match['home_team_id']: h2h_home += 1
                else: h2h_away += 1
            elif winner == 2:
                if h2h_home_id == match['home_team_id']: h2h_away += 1
                else: h2h_home += 1
            elif winner == 3:
                h2h_draws += 1
        match['h2h_home_wins'] = h2h_home
        match['h2h_away_wins'] = h2h_away
        match['h2h_draws'] = h2h_draws
    
    return match

print('Full scan functions ready')

In [ ]:
#@title 🚀 Run Full Week Scan { display-mode: "form" }

all_new = []
start_time = time.time()

print('='*60)
print('INCREMENTAL UPDATE v3 - FULL WEEK SCAN')
print('='*60)
print(f'Known match IDs: {len(existing_ids):,}')
print(f'Checking {Pages_To_Check} pages per league (~1 week of matches)\n')

for name, info in selected.items():
    print(f'🔍 {name}')
    
    for tid in info['ids']:
        matches = get_finished_matches_full_scan(tid, existing_ids, Pages_To_Check)
        
        if matches:
            print(f'  ✅ Found {len(matches)} NEW matches\n')
            for m in matches:
                m['league'] = name
            all_new.extend(matches)
            break
        else:
            print(f'  ⚪ No new matches\n')

print(f'📊 Total NEW matches: {len(all_new)}')

if all_new:
    print(f'\n⚙️ Enriching {len(all_new)} matches...')
    enriched = []
    with ThreadPoolExecutor(max_workers=Parallel_Workers) as executor:
        futures = [executor.submit(enrich_match, m) for m in all_new]
        for i, future in enumerate(as_completed(futures)):
            enriched.append(future.result())
            print(f'\r  Progress: {i+1}/{len(all_new)}', end='', flush=True)
    
    df_new = pd.DataFrame(enriched)
    print(f'\n\n✅ Enriched {len(df_new)} matches')
    print(f'⏱️ Time: {(time.time() - start_time)/60:.1f} minutes')
    
    print('\n📋 New matches preview:')
    display(df_new[['date', 'league', 'home_team', 'away_team', 'home_score', 'away_score']].head(20))
else:
    df_new = pd.DataFrame()
    print('\n✅ No gaps found - your data is complete!')

In [ ]:
#@title 📥 Download Updated Data { display-mode: "form" }

if len(df_new) > 0:
    df_combined = pd.concat([df_existing, df_new], ignore_index=True)
    df_combined = df_combined.drop_duplicates(subset=['match_id'], keep='last')
    df_combined = df_combined.sort_values('date').reset_index(drop=True)
    
    today = datetime.now().strftime('%Y%m%d')
    out_filename = f'sofascore_updated_{len(df_combined)}matches_{today}.csv'
    df_combined.to_csv(out_filename, index=False)
    
    print('='*60)
    print('UPDATE COMPLETE!')
    print('='*60)
    print(f'Previous: {len(df_existing):,} matches')
    print(f'Added:    {len(df_new):,} new matches')
    print(f'Total:    {len(df_combined):,} matches')
    print(f'\n📅 Date range: {df_combined["date"].min()} to {df_combined["date"].max()}')
    
    files.download(out_filename)
else:
    print('No new data to download.')

---
## v3 Changes
- **Full scan**: Checks ALL pages up to limit (no early stopping)
- **Configurable**: Slider to set pages (20 = ~1 week per league)
- **Gap detection**: Finds missing matches even if surrounded by known ones